In [1]:
import marqo
import json

mq = marqo.Client(url='http://localhost:8882')
index_name = "book-index"

# Check if this index has been created, if so, delete
try:
    mq.index(index_name).delete()
except:
    pass

mq.create_index(index_name, model="hf/e5-base-v2")

# Load pre-cleaned data
with open(r'docs\QGenda Whitepaper_cleaned.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Index to marqo
mq.index(index_name).add_documents(
    data,
    tensor_fields=["text"]
)

{'errors': False,
 'processingTimeMs': 4635.589048999464,
 'index_name': 'book-index',
 'items': [{'status': 200, '_id': '4cff9e32-e123-47e9-8ea5-501d1824a8c8'},
  {'status': 200, '_id': '7579c148-9a56-448e-8ad1-4b0358c35e76'},
  {'status': 200, '_id': '3a0a46cc-69ba-484f-8fe4-6ed0564f896f'},
  {'status': 200, '_id': '7538b76f-ecfc-4c92-99e8-c0353adcf95b'},
  {'status': 200, '_id': 'cc09dc29-ead2-41c9-994d-4113aead5873'},
  {'status': 200, '_id': 'c78cb57a-39f0-4f2c-95b8-9821d589ba25'}]}

In [2]:
# Perform generic search query
question="What utilization gap do executives need to focus on?"

results = mq.index("book-index").search(
    q=question,
    limit=2
)
print(results)

{'hits': [{'index': 1, '_id': '7579c148-9a56-448e-8ad1-4b0358c35e76', 'text': 'As healthcare leaders reimagine patient access and care delivery in a post-pandemic world, there is an emerging story around the need to optimize clinic exam room utilization. Better utilization can improve operating efficiency, patient satisfaction, and revenue capture. The 2020 Porter Research study of 100 health system executive leaders identified the many challenges faced today with exam room scheduling, future expectations for optimizing exam rooms, and the impact that proper exam room scheduling can have on a health system’s P&L. Increasing Scrutiny on Clinic Exam Rooms With the limited funds available for health system capital expenditures, executives must optimize their existing physical space while simultaneously remaining adaptable to fluctuations in patient demand. In the short term, health systems may experience an influx of patients who delayed elective surgeries, treatments for minor health iss

In [3]:
import os

openai_key = os.getenv('OPENAI_API_KEY')

if openai_key:
    print("OpenAI key accessed.")
else:
    print("API Key not found. Check your environment variables.")

OpenAI key accessed.


In [4]:
# Set up prompt for GPT
from langchain.prompts import PromptTemplate

template = """
Given the following extracted parts of a long document ("SOURCES") and a question ("QUESTION"), create a final answer one paragraph long.
Don't try to make up an answer and use the text in the SOURCES only for the answer. If you don't know the answer, just say that you don't know.
QUESTION: {question}
=========
SOURCES:
{summaries}
=========
ANSWER:
"""
prompt = PromptTemplate(template=template, input_variables=["summaries", "question"])

In [5]:
# Run LLM
from langchain_openai import OpenAI
from langchain.docstore.document import Document
from langchain.chains import LLMChain
from utilities import extract_text_from_highlights

highlights, texts = extract_text_from_highlights(results, token_limit=150)
docs = [Document(page_content=f"Source [{ind}]:" + t) for ind, t in enumerate(texts)]
llm = OpenAI(temperature=0.9)
chain_qa = LLMChain(llm=llm, prompt=prompt)
llm_results = chain_qa.invoke({"summaries": docs, "question": results['query']}, return_only_outputs=True)
print(llm_results['text'])

C:\Users\trott\AppData\Local\Temp\ipykernel_11220\3493308595.py:10: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain_qa = LLMChain(llm=llm, prompt=prompt)


Executives need to focus on closing a 20-point utilization gap, as identified by health system executives surveyed. This gap refers to the difference between the optimal exam room utilization of 80-89% and the current utilization rate. While it may be challenging, small improvements in utilization can result in significant cost savings, according to a health system that saw a $140,000 per year increase for every 1% improvement in utilization. Additionally, 70% of executives surveyed expressed interest in solutions that can help close this gap, as proper exam room management is crucial but can be difficult without purpose-built technology.
